### ResNet18 Trained

### Initialize Notebook & packages

In [1]:
import helper as hp 
hp.initialize_notebook() # initialize with GPU enabled  
# hp.initialize_notebook(False) # to disable GPU 

2024-12-15 13:06:41.329264: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 13:06:41.367679: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 13:06:41.377383: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 13:06:41.675334: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


GPU enabled. Checking for available GPUs...
1 Physical GPUs, 1 Logical GPUs

Verifying TensorFlow and PyTorch CUDA setup...
TensorFlow version: 2.17.0
Built with CUDA: True
Num GPUs Available: 1

Keras version: 3.6.0

End checks and initialization.


I0000 00:00:1734268014.160985  667231 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734268014.969140  667231 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734268014.974488  667231 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734268014.981592  667231 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [2]:
# Import standard libraries
import os
import sys
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import keras
# Import DL libraries
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import layers, models, Model, Input
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.callbacks import TensorBoard 

# Suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Import ML libraries
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

# Import image libraries
from skimage import transform
import tifffile as tiff

# Appends current working dir
current_path = os.getcwd()
sys.path.append(current_path)

# Import custom preprocessing class
from imc_preprocessing import IMCPreprocessor

# Import Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

#### Methods

In [3]:
# Preprocessing (if needed)
def preprocessing(image, transpose=True, normalize=True) -> np.ndarray:
    if transpose:
        return np.transpose(image, (1, 2, 0))
    if normalize:
        return IMCPreprocessor.normalize_multichannel_image(image)

# Load images
def load_image(image_path) -> np.ndarray:
    image = tiff.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")
    return image


# Define a function to create a list of images from files within a folder 
def image_list(image_dir):
    # List all files in the directory
    image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]  
    # Initialize a list to store the images
    images = []
    image_files_list = []
    # Loop through each file and read the image
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        image = load_image(image_path)
        images.append(image)  
        image_files_list.append(image_file)
    return [images, image_files_list] 

def image_list_and_PDL1(image_dir):
    # List all files in the directory
    image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]  
    # Initialize a list to store the images
    images = []
    PDL1 = []
    image_files_list = []
    # Loop through each file and read the image
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        image = load_image(image_path)
        images.append(image)  
        image_files_list.append(image_file)
        PDL1.append(metadata[metadata["sample_id"] == image_file.strip(".tiff")]["PDL1_score"])
    return [images, image_files_list, PDL1] 

# Converting to one hot
def convert_to_one_hot(y, classes):
    return np.eye(classes)[y]

### Data 

#### Preprocessing and Other

In [4]:

# images_dir = '/home/jupyter-luvogt/Final_Project_LR/IMC_images' 
# metadata_dir = '/home/jupyter-luvogt/Final_Project_LR/metadata.csv' 
# panel_dir = '/home/jupyter-luvogt/Final_Project_LR/panel.csv' 
# os.listdir(images_dir)[:5] # Get first five images

# # Load images
# images = image_list(images_dir)[0]
# images = np.array(images)

# # Load Image_paths
# image_files = image_list(images_dir)[1] # Define image files for debugging
# image_files = np.array(image_files)
# # load labels
# metadata = pd.read_csv(metadata_dir)
# PDL1_score = metadata["PDL1_score"]

# # Shape PDL1
# PDL1_score = PDL1_score.tolist()
# PDL1_score = np.array(PDL1_score)

# # Transpose and Normalize images
# images_preproc = [preprocessing(i, transpose = True, normalize = False) for i in images]
# images_preproc = [preprocessing(i, transpose = False, normalize = True) for i in images_preproc]
# images_preproc = np.array(images_preproc)

# # Extract channel information
# panel_df = pd.read_csv(panel_dir)
# channel_names = dict(zip(panel_df['clean_target'].to_list(), panel_df['channel'].to_list()))

# # Filter out Xe131, Xe134 and Ba138 = Noise channels (OPTIONAL) 
# channel_names_new = [x for x in list(channel_names.values()) if x not in ["Xe131", "Xe134", "Ba138"]]
# images_preproc_drop = [IMCPreprocessor.drop_channels(i, channel_names_new, list(channel_names.values()))[0] for i in images_preproc]
# images_preproc_drop = np.array(images_preproc_drop)

### ResNet18 Model: Trained directly here

#### ResNet18 Model: 3 Channels

Approach: Select biological relevant channels that correspond or are associated with PDL1 

In [5]:
# # Choose 3 biological relevant channels
# channel_names_new = ["Gd160", "Eu153", "Gd155"]
# images_preproc_drop_3 = [IMCPreprocessor.drop_channels(i, channel_names_new, list(channel_names.values()))[0] for i in images_preproc]
# images_preproc_drop_3 = np.array(images_preproc_drop_3)

# channels_preproc_drop_3 = channel_names_new

Create unbalanced (but with stratified) training, validation and test set

In [6]:
# random_seed = 56
# X = images_preproc_drop # Change here if you want 43 channels or 3 channels (images_preproc_drop_3)
# y = PDL1_score
# train_size = 0.6
# val_size = 0.2
# test_size = 0.2


# # Create a StratifiedShuffleSplit for train/test split
# sss_train_test = StratifiedShuffleSplit(n_splits=1, test_size=(val_size + test_size), random_state=random_seed)

# # First split: Train and remaining (validation + test)
# for train_index, remaining_index in sss_train_test.split(X, y):
#     X_train, X_remaining = X[train_index], X[remaining_index]
#     y_train, y_remaining = y[train_index], y[remaining_index]

# # Create a StratifiedShuffleSplit for validation/test split on remaining data
# sss_val_test = StratifiedShuffleSplit(n_splits=1, test_size=test_size / (val_size + test_size), random_state=random_seed)

# # Second split: Validation and Test
# for val_index, test_index in sss_val_test.split(X_remaining, y_remaining):
#     X_val, X_test = X_remaining[val_index], X_remaining[test_index]
#     y_val, y_test = y_remaining[val_index], y_remaining[test_index]


Import One Hot Encoding

In [6]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.utils import plot_model
# from keras.utils.vis_utils import model_to_dot
import keras.backend as K
import tensorflow as tf
# CLASSES = 2
# y_train_one_hot = convert_to_one_hot(y_train, CLASSES)
# y_test_one_hot = convert_to_one_hot(y_test, CLASSES)
# y_val_one_hot = convert_to_one_hot(y_val, CLASSES)

Define Blocks and ResNet Model

In [5]:
from keras.regularizers import l2
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

def ResNet(shape = (32, 32, 3), classes = 10, block_layers = [3, 4, 6, 3]):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = block_layers
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

Build Model

In [11]:
# ROWS = 224
# COLS = 224
# CHANNELS = 43
# CLASSES = 2
# block_layers = [2,2,2,2]
# # Build Network Graph 
# model_ResNet18 = ResNet(shape = (ROWS, COLS, CHANNELS), classes = CLASSES, block_layers = block_layers)

# # Compile Model 
# l_rate = 1.e-4
# opt = keras.optimizers.Adam(learning_rate=l_rate)

# model_ResNet18.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# # # Apply TensorBoard
# # # define the logs folder 
# # log_dir = os.path.join("logs_ResNet50", "fit", "model_ResNet50_3Channels" + time.strftime("%Y%m%d-%H%M%S"))
# # # Define TensorBoard Callback
# # tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# # Train Model 
# batch = 32
# epochs = 10
# start_time = time.time()

# history_ResNet18 = model_ResNet18.fit(X_train, y_train_one_hot, 
#                                       epochs = epochs, batch_size = batch, 
#                                       validation_data = (X_val, y_val_one_hot))

# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"\nElapsed time: {elapsed_time} seconds")

KeyboardInterrupt: 

In [38]:
# t = model_ResNet18.predict(X_val)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step


In [28]:
# model_ResNet18.evaluate(X_train, y_train_one_hot)

19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.6339 - loss: 0.7278


[0.672294557094574, 0.6593220233917236]

#### COMMENTS

Even with ResNet18, the model still seems to only predict the majority class. Thus, in order to try to address this issue, we will first attack the problem from the unbalanced data set problem side: 

    - Introduce Class Weights
    - Create Balanced data set
    - Increase minority class with data augmentation

### ResNet10: Introducing Class Weights

Model Class Weights Calculation

And even reduce Model size to ResNet10

In [33]:
# from sklearn.utils import class_weight
# weights = class_weight.compute_class_weight("balanced", classes = np.unique(y_train), y = y_train)
# class_weights = dict(enumerate(weights))

In [40]:
# ROWS = 224
# COLS = 224
# CHANNELS = 43
# CLASSES = 2
# block_layers = [1,1,1,1]
# # Build Network Graph 
# model_ResNet18 = ResNet(shape = (ROWS, COLS, CHANNELS), classes = CLASSES, block_layers = block_layers)

# # Compile Model 
# l_rate = 1.e-4
# opt = keras.optimizers.Adam(learning_rate=l_rate)

# model_ResNet18.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# # # Apply TensorBoard
# # # define the logs folder 
# # log_dir = os.path.join("logs_ResNet50", "fit", "model_ResNet50_3Channels" + time.strftime("%Y%m%d-%H%M%S"))
# # # Define TensorBoard Callback
# # tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# # Train Model 
# batch = 32
# epochs = 10
# start_time = time.time()

# history_ResNet18 = model_ResNet18.fit(X_train, y_train_one_hot, 
#                                       epochs = epochs, batch_size = batch, 
#                                       validation_data = (X_val, y_val_one_hot), 
#                                       class_weight = class_weights)

# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"\nElapsed time: {elapsed_time} seconds")

Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 28s 782ms/step - accuracy: 0.4883 - loss: 1.0176 - val_accuracy: 0.3604 - val_loss: 0.7011
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 160ms/step - accuracy: 0.5340 - loss: 0.7055 - val_accuracy: 0.6396 - val_loss: 0.6847
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 160ms/step - accuracy: 0.6233 - loss: 0.6568 - val_accuracy: 0.6396 - val_loss: 0.6741
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 160ms/step - accuracy: 0.6122 - loss: 0.6719 - val_accuracy: 0.6396 - val_loss: 0.6804
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 161ms/step - accuracy: 0.6652 - loss: 0.6170 - val_accuracy: 0.6396 - val_loss: 0.6734
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 162ms/step - accuracy: 0.7744 - loss: 0.5516 - val_accuracy: 0.6396 - val_loss: 0.6698
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - accuracy: 0.8152 - loss: 0.5221 - val_accuracy: 0.6396 - val_loss: 0.6656
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 161ms/step - accuracy: 0.8035 - loss: 0.4452 - val_accuracy: 0

In [41]:
# t = model_ResNet18.predict(X_test)
# print(t[1:100])

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 154ms/step
[[0.9151241  0.08487583]
 [0.8824722  0.11752776]
 [0.9190049  0.08099506]
 [0.8523949  0.14760512]
 [0.903726   0.09627395]
 [0.8987082  0.1012918 ]
 [0.87999773 0.12000225]
 [0.92008114 0.07991882]
 [0.90192103 0.09807896]
 [0.9014896  0.09851039]
 [0.8976786  0.10232136]
 [0.87083733 0.12916267]
 [0.8840908  0.1159092 ]
 [0.90777695 0.09222302]
 [0.9016646  0.09833534]
 [0.88322604 0.11677395]
 [0.89981246 0.10018753]
 [0.86775917 0.1322408 ]
 [0.8882554  0.11174451]
 [0.8954403  0.10455975]
 [0.85610664 0.14389338]
 [0.8988902  0.10110982]
 [0.8975536  0.10244637]
 [0.8974461  0.10255393]
 [0.88729185 0.11270811]
 [0.9015776  0.09842241]
 [0.8887577  0.11124229]
 [0.9019362  0.09806384]
 [0.89208215 0.10791782]
 [0.8880185  0.11198147]
 [0.89878786 0.10121211]
 [0.8891496  0.11085043]
 [0.893966   0.10603395]
 [0.8838623  0.11613767]
 [0.904973   0.09502705]
 [0.8793694  0.12063062]
 [0.90389097 0.09610905]
 [0.8961733  0.10382664]
 [0.8936265

#### COMMENTS

    - Introducing Class Weights: No difference, model overfits and only predicts majority class, even with ResNet10 (less complex model)
    - NOW: Try Balanced Dataset

### ResNet10: Downsampling of Majority Class

Downsampling of Majority Class 

In [8]:
# import imblearn
# from imblearn.under_sampling import RandomUnderSampler

# # Define Undersampling balancing method
# balancer = RandomUnderSampler(random_state = 42) # Undersampling majority class

# images_flat = images_preproc_drop.reshape(images_preproc_drop.shape[0], -1) # Reshape images for balancer
# images_preproc_drop_resampled, PDL1_resampled = balancer.fit_resample(images_flat, PDL1_score) # resample (Undersampling)

# images_preproc_drop_resampled = images_preproc_drop_resampled.reshape(-1, *images_preproc_drop.shape[1:]) # Re shape the image

/home/jupyter-luvogt/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-luvogt/.local/lib/python3.12/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/home/jupyter-luvogt/.local/lib/python3.12/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [9]:
# random_seed = 56
# X = images_preproc_drop_resampled # Change here if you want 43 channels or 3 channels (images_preproc_drop_3)
# y = PDL1_resampled
# train_size = 0.6
# val_size = 0.2
# test_size = 0.2


# # Create a StratifiedShuffleSplit for train/test split
# sss_train_test = StratifiedShuffleSplit(n_splits=1, test_size=(val_size + test_size), random_state=random_seed)

# # First split: Train and remaining (validation + test)
# for train_index, remaining_index in sss_train_test.split(X, y):
#     X_train, X_remaining = X[train_index], X[remaining_index]
#     y_train, y_remaining = y[train_index], y[remaining_index]

# # Create a StratifiedShuffleSplit for validation/test split on remaining data
# sss_val_test = StratifiedShuffleSplit(n_splits=1, test_size=test_size / (val_size + test_size), random_state=random_seed)

# # Second split: Validation and Test
# for val_index, test_index in sss_val_test.split(X_remaining, y_remaining):
#     X_val, X_test = X_remaining[val_index], X_remaining[test_index]
#     y_val, y_test = y_remaining[val_index], y_remaining[test_index]

# CLASSES = 2
# y_train_one_hot = convert_to_one_hot(y_train, CLASSES)
# y_test_one_hot = convert_to_one_hot(y_test, CLASSES)
# y_val_one_hot = convert_to_one_hot(y_val, CLASSES)


In [10]:
# ROWS = 224
# COLS = 224
# CHANNELS = 43
# CLASSES = 2
# block_layers = [1,1,1,1]
# # Build Network Graph 
# model_ResNet18 = ResNet(shape = (ROWS, COLS, CHANNELS), classes = CLASSES, block_layers = block_layers)

# # Compile Model 
# l_rate = 1.e-4
# opt = keras.optimizers.Adam(learning_rate=l_rate)

# model_ResNet18.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# # # Apply TensorBoard
# # # define the logs folder 
# # log_dir = os.path.join("logs_ResNet50", "fit", "model_ResNet50_3Channels" + time.strftime("%Y%m%d-%H%M%S"))
# # # Define TensorBoard Callback
# # tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# # Train Model 
# batch = 32
# epochs = 10
# start_time = time.time()

# history_ResNet18 = model_ResNet18.fit(X_train, y_train_one_hot, 
#                                       epochs = epochs, batch_size = batch, 
#                                       validation_data = (X_val, y_val_one_hot))

# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"\nElapsed time: {elapsed_time} seconds")

2024-12-14 19:43:12.846543: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3685126144 exceeds 10% of free system memory.
2024-12-14 19:43:14.582333: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3685126144 exceeds 10% of free system memory.


Epoch 1/10


I0000 00:00:1734205401.171647  652570 service.cc:146] XLA service 0x7fdf78003800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734205401.171703  652570 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-12-14 19:43:21.307176: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-14 19:43:21.893100: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90300
I0000 00:00:1734205412.392863  652570 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


14/14 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.4951 - loss: 1.1123 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 155ms/step - accuracy: 0.8633 - loss: 0.4701 - val_accuracy: 0.5000 - val_loss: 0.6937
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 157ms/step - accuracy: 0.9943 - loss: 0.2471 - val_accuracy: 0.5000 - val_loss: 0.6940
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 153ms/step - accuracy: 1.0000 - loss: 0.1279 - val_accuracy: 0.4789 - val_loss: 0.6933
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 154ms/step - accuracy: 1.0000 - loss: 0.0508 - val_accuracy: 0.5000 - val_loss: 0.6981
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 156ms/step - accuracy: 1.0000 - loss: 0.0263 - val_accuracy: 0.5000 - val_loss: 0.7128
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 155ms/step - accuracy: 1.0000 - loss: 0.0161 - val_accuracy: 0.5000 - val_loss: 0.7075
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 154ms/step - accuracy: 1.0000 - loss: 0.0103 - val_accuracy: 0.5000 - val_lo

In [21]:
# model_ResNet18.predict(X_test)[1:20] # Only Predicts PDL1 == 0

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


array([[0.7666656 , 0.23333445],
       [0.7796348 , 0.22036527],
       [0.79146713, 0.20853293],
       [0.79206043, 0.20793962],
       [0.75556463, 0.24443538],
       [0.78355   , 0.21644998],
       [0.7772638 , 0.22273616],
       [0.7973107 , 0.20268928],
       [0.7658215 , 0.23417851],
       [0.77662545, 0.2233745 ],
       [0.8071066 , 0.1928934 ],
       [0.7514903 , 0.24850976],
       [0.79176426, 0.20823574],
       [0.8236233 , 0.17637675],
       [0.7840638 , 0.21593621],
       [0.7984144 , 0.20158562],
       [0.78082216, 0.21917789],
       [0.7725018 , 0.22749814],
       [0.8092341 , 0.19076596]], dtype=float32)

In [20]:
# model_ResNet18.evaluate(X_test[0:5], y_test_one_hot[0:5]) # 0 Accuracy for PDL1 == 1

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 1.0788


[1.0788371562957764, 0.0]

#### COMMENTS

Still not improvement: even with balanced dataset, model just predicts one dataset...

Until know, we tried to adress the problem as follows: 

    - Choose biological relevant channels
    - Increase number of channels
    - Introduce Dropout / Regularization
    - Decrease model complexity: ResNet50 --> ResNet18 --> ResNet 10 
    - Balanced dataset
    --> Still: model only predicts one class in our binary classification task

    Next Steps to take: 
    - Verify label encoding
    - Increase model complexity for ResNet
    - Dimensionality Reduction 
    --> If all these steps don't deliver any improvements, then: 
    - Change Model Architecture

### Verifying Label Encoding

In [9]:
# Analyze Image Paths

print(image_files[0:5]) 
print(PDL1_score[0:5])

['ZTMA224.1_BlockB_SE_123.tiff' 'ZTMA224.1_BlockC_SE_098.tiff'
 'ZTMA224.1_BlockB_SE_020.tiff' 'ZTMA20.4_Block1_SE_012.tiff'
 'ZTMA224.1_BlockA_SE_003.tiff']
[1 1 1 0 0]


#### COMMENTS

It seems very likely that the function image_list doesn't follow the order of the metadata.csv data as PDL1_score --> get wrong label encodings...

Adjust import of images in such a way that in imports it in accordance with the PDL1 Score

Idea: Import images based on image_file annotation (= sample_id) --> define image_list_and_PDL1()

In [7]:
images_dir = '/home/jupyter-luvogt/Final_Project_LR/IMC_images' 
metadata_dir = '/home/jupyter-luvogt/Final_Project_LR/metadata.csv'
panel_dir = '/home/jupyter-luvogt/Final_Project_LR/panel.csv' 
metadata = pd.read_csv(metadata_dir)

# Load images CORRECT WITH image_list_and_PDL1
images = image_list_and_PDL1(images_dir)[0]
images = np.array(images)

# Load image files names

image_files = image_list_and_PDL1(images_dir)[1] # Define image files for debugging
image_files = np.array(image_files)
# Load PDL1 to match images --> correct label encoding
PDL1_score = image_list_and_PDL1(images_dir)[2] # Define image files for debugging
PDL1_score = np.array(PDL1_score)

# Transpose and Normalize images
images_preproc = [preprocessing(i, transpose = True, normalize = False) for i in images]
images_preproc = [preprocessing(i, transpose = False, normalize = True) for i in images_preproc]
images_preproc = np.array(images_preproc)

# Extract channel information
panel_df = pd.read_csv(panel_dir)
channel_names = dict(zip(panel_df['clean_target'].to_list(), panel_df['channel'].to_list()))

# Filter out Xe131, Xe134 and Ba138 = Noise channels (OPTIONAL) 
channel_names_new = [x for x in list(channel_names.values()) if x not in ["Xe131", "Xe134", "Ba138"]]
images_preproc_drop = [IMCPreprocessor.drop_channels(i, channel_names_new, list(channel_names.values()))[0] for i in images_preproc]
images_preproc_drop = np.array(images_preproc_drop)

Veryifing correct Label Encoding

In [14]:
print(image_files[0:20])

print(PDL1_score[0:20].tolist())
# Confirming label encoding
test = [i.strip(".tiff") for i in image_files[0:20]]
for i in test:
    print(metadata[metadata["sample_id"] == i]["PDL1_score"].tolist()[0])

['ZTMA224.1_BlockB_SE_123.tiff' 'ZTMA224.1_BlockC_SE_098.tiff'
 'ZTMA224.1_BlockB_SE_020.tiff' 'ZTMA20.4_Block1_SE_012.tiff'
 'ZTMA224.1_BlockA_SE_003.tiff' 'ZTMA20.4_Block2_SE_051.tiff'
 'ZTMA20.1_Block3_SE_114.tiff' 'ZTMA224.1_BlockC_SE_107.tiff'
 'ZTMA224.2_BlockC_121.tiff' 'ZTMA224.1_BlockA_SE_012.tiff'
 'ZTMA224.2_BlockB_073.tiff' 'ZTMA224.1_BlockB_SE_084.tiff'
 'ZTMA224.2_BlockB_081.tiff' 'ZTMA20.1_Block2_SE_108.tiff'
 'ZTMA224.2_BlockA_082.tiff' 'ZTMA20.1_Block3_SE_056.tiff'
 'ZTMA224.2_BlockC_120.tiff' 'ZTMA224.2_BlockC_089.tiff'
 'ZTMA224.2_BlockC_037.tiff' 'ZTMA20.1_Block2_SE_055.tiff']
[[0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0]]
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0


#### COMMENTS

Labeling seems to be correct now --> Restart with ResNet10 for proof of concept

Training, validation and test split: Downsampling to balanced dataset

In [8]:
random_seed = 56
X = image_files # Change here if you want 43 channels or 3 channels (images_preproc_drop_3)
y = PDL1_score
train_size = 0.6; val_size = 0.2; test_size = 0.2
# Create a StratifiedShuffleSplit for train/test split
sss_train_test = StratifiedShuffleSplit(n_splits=1, test_size=(val_size + test_size), random_state=random_seed)
# First split: Train and remaining (validation + test)
for train_index, remaining_index in sss_train_test.split(X, y):
    X_train, X_remaining = X[train_index], X[remaining_index]
    y_train, y_remaining = y[train_index], y[remaining_index]
# Create a StratifiedShuffleSplit for validation/test split on remaining data
sss_val_test = StratifiedShuffleSplit(n_splits=1, test_size=test_size / (val_size + test_size), random_state=random_seed)
# Second split: Validation and Test
for val_index, test_index in sss_val_test.split(X_remaining, y_remaining):
    X_val, X_test = X_remaining[val_index], X_remaining[test_index]
    y_val, y_test = y_remaining[val_index], y_remaining[test_index]

CLASSES = 2
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)
y_val_one_hot = to_categorical(y_val)

Test whether Label encoding correct

In [24]:
print(X_val[0:20])
print(y_val_one_hot[0:20].tolist())
# Confirming label encoding
test = [i.strip(".tiff") for i in X_val[0:20]]
for i in test:
    print(metadata[metadata["sample_id"] == i]["PDL1_score"].tolist()[0], end = " ")

['ZTMA20.1_Block2_SE_121.tiff' 'ZTMA20.4_Block1_SE_075.tiff'
 'ZTMA224.2_BlockB_077.tiff' 'ZTMA224.2_BlockC_128.tiff'
 'ZTMA224.1_BlockA_SE_028.tiff' 'ZTMA20.1_Block1_SE_085.tiff'
 'ZTMA224.2_BlockC_043.tiff' 'ZTMA20.4_Block1_SE_121.tiff'
 'ZTMA224.2_BlockC_120.tiff' 'ZTMA224.2_BlockA_014.tiff'
 'ZTMA224.1_BlockA_SE_031.tiff' 'ZTMA224.2_BlockA_032.tiff'
 'ZTMA224.1_BlockC_SE_043.tiff' 'ZTMA20.4_Block1_SE_092.tiff'
 'ZTMA20.4_Block2_SE_038.tiff' 'ZTMA224.1_BlockB_SE_064.tiff'
 'ZTMA224.1_BlockC_SE_041.tiff' 'ZTMA20.1_Block1_SE_026.tiff'
 'ZTMA20.4_Block3_SE_074.tiff' 'ZTMA20.1_Block2_SE_070.tiff']
[[0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0]]
1 1 0 1 0 1 0 1 0 0 1 1 0 1 0 0 0 0 0 1 

Retranslate image file names to images themselves

Reason: to make sure that Label Encoding stays intact!

In [9]:
X_train_array = np.zeros((len(X_train), 224, 224, 43), dtype=np.float32)
X_test_array = np.zeros((len(X_test), 224, 224, 43), dtype=np.float32)
X_val_array = np.zeros((len(X_val), 224, 224, 43), dtype=np.float32)

# CHANGE HERE
# X_train_array = np.zeros((len(X_train), 224, 224, 3), dtype=np.float32)
# X_test_array = np.zeros((len(X_test), 224, 224, 3), dtype=np.float32)
# X_val_array = np.zeros((len(X_val), 224, 224, 3), dtype=np.float32)


In [10]:
channel_names_new = [x for x in list(channel_names.values()) if x not in ["Xe131", "Xe134", "Ba138"]]
# channel_names_new = ["Gd160", "Eu153", "Gd155"] # CHANGE HERE 
for i, x in enumerate(X_train): 
    image = load_image(os.path.join(images_dir, x))
    image = preprocessing(image, transpose = True, normalize = False)
    image = preprocessing(image, transpose = False, normalize = True)
    image = IMCPreprocessor.drop_channels(image, channel_names_new, list(channel_names.values()))[0]
    X_train_array[i] = image

for i, x in enumerate(X_test): 
    image = load_image(os.path.join(images_dir, x))
    image = preprocessing(image, transpose = True, normalize = False)
    image = preprocessing(image, transpose = False, normalize = True)
    image = IMCPreprocessor.drop_channels(image, channel_names_new, list(channel_names.values()))[0]
    X_test_array[i] = image

for i, x in enumerate(X_val): 
    image = load_image(os.path.join(images_dir, x))
    image = preprocessing(image, transpose = True, normalize = False)
    image = preprocessing(image, transpose = False, normalize = True)
    image = IMCPreprocessor.drop_channels(image, channel_names_new, list(channel_names.values()))[0]
    X_val_array[i] = image

In [58]:
print(X_train_array.shape)
print(X_test_array.shape)
print(X_val_array.shape)

(590, 224, 224, 43)
(197, 224, 224, 43)
(197, 224, 224, 43)


#### Build ResNet10: Correct Label Encoding

In [36]:
ROWS = 224; COLS = 224; CHANNELS = 43; CLASSES = 2 # CHANGE HERE
block_layers = [1,1,1,1]
# Build Network Graph 
model_ResNet10 = ResNet(shape = (ROWS, COLS, CHANNELS), classes = CLASSES, block_layers = block_layers)
# Compile Model 
l_rate = 1.e-4
opt = keras.optimizers.Adam(learning_rate=l_rate)
model_ResNet10.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
# Train Model 
batch = 32; epochs = 10
start_time = time.time()
history_ResNet10 = model_ResNet10.fit(X_train_array, y_train_one_hot, 
                                      epochs = epochs, batch_size = batch,
                                     validation_data = (X_val_array, y_val_one_hot))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nElapsed time: {elapsed_time} seconds")

Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 23s 637ms/step - accuracy: 0.5725 - loss: 0.9742 - val_accuracy: 0.6396 - val_loss: 0.6693
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 169ms/step - accuracy: 0.7946 - loss: 0.4411 - val_accuracy: 0.6396 - val_loss: 0.6554
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - accuracy: 0.9198 - loss: 0.2480 - val_accuracy: 0.6396 - val_loss: 0.6526
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - accuracy: 0.9988 - loss: 0.1251 - val_accuracy: 0.6396 - val_loss: 0.6580
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - accuracy: 1.0000 - loss: 0.0516 - val_accuracy: 0.6396 - val_loss: 0.6644
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 166ms/step - accuracy: 1.0000 - loss: 0.0218 - val_accuracy: 0.6396 - val_loss: 0.6714
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step - accuracy: 1.0000 - loss: 0.0148 - val_accuracy: 0.6396 - val_loss: 0.7431
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 167ms/step - accuracy: 1.0000 - loss: 0.0068 - val_accuracy: 0

#### COMMENTS

Even with correct Label Encoding, model still only predicts one class

    - Try with balanced dataset again

Balanced Data Set

In [11]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler

# X_train: Define Undersampling balancing method
balancer = RandomUnderSampler(random_state = 42) # Undersampling majority class
X_train_array_flat = X_train_array.reshape(X_train_array.shape[0], -1) # Reshape images for balancer
X_train_array_resampled, y_train_resampled = balancer.fit_resample(X_train_array_flat, y_train) # resample (Undersampling)
X_train_array_resampled = X_train_array_resampled.reshape(-1, *X_train_array.shape[1:]) # Reshape the image

# X_val: Define Undersampling balancing method
balancer = RandomUnderSampler(random_state = 42) # Undersampling majority class
X_val_array_flat = X_val_array.reshape(X_val_array.shape[0], -1) # Reshape images for balancer
X_val_array_resampled, y_val_resampled = balancer.fit_resample(X_val_array_flat, y_val) # resample (Undersampling)
X_val_array_resampled = X_val_array_resampled.reshape(-1, *X_val_array.shape[1:]) # Reshape the image


/home/jupyter-luvogt/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-luvogt/.local/lib/python3.12/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/home/jupyter-luvogt/.local/lib/python3.12/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/home/jupyter-luvogt/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-luvogt/.local/lib/python3.12/site-packages/sklearn/bas

In [84]:
print(X_train_array_resampled.shape)
y_train_resampled

(426, 224, 224, 43)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Reshuffle labels and X_training_array_resampled

In [13]:
from sklearn.utils import shuffle

# X_train: Shuffle the data and labels together
X_train_array_resampled, y_train_resampled = shuffle(X_train_array_resampled, y_train_resampled, random_state=42)
y_train_resampled_one_hot = to_categorical(y_train_resampled)

# X_val: Shuffle the data and labels together
X_val_array_resampled, y_val_resampled = shuffle(X_val_array_resampled, y_val_resampled, random_state=42)
y_val_resampled_one_hot = to_categorical(y_val_resampled)

#### Build ResNet10: Correct Label Encoding + downsampling 

In [40]:
ROWS = 224; COLS = 224; CHANNELS = 43; CLASSES = 2 # CHANGE HERE
block_layers = [1,1,1,1]
# Build Network Graph 
model_ResNet10 = ResNet(shape = (ROWS, COLS, CHANNELS), classes = CLASSES, block_layers = block_layers)
# Compile Model 
l_rate = 1.e-4
opt = keras.optimizers.Adam(learning_rate=l_rate)
model_ResNet10.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
# Train Model 
batch = 64; epochs = 10
start_time = time.time()
history_ResNet10 = model_ResNet10.fit(X_train_array_resampled, y_train_resampled_one_hot, 
                                      epochs = epochs, batch_size = batch,
                                     validation_data = (X_val_array_resampled, y_val_resampled_one_hot))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nElapsed time: {elapsed_time} seconds")

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5665 - loss: 0.9178 - val_accuracy: 0.5000 - val_loss: 0.6927
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 326ms/step - accuracy: 0.8440 - loss: 0.4000 - val_accuracy: 0.5000 - val_loss: 0.6974
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 325ms/step - accuracy: 0.9202 - loss: 0.2447 - val_accuracy: 0.5000 - val_loss: 0.7015
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 326ms/step - accuracy: 0.9726 - loss: 0.1184 - val_accuracy: 0.5000 - val_loss: 0.7007
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 326ms/step - accuracy: 0.9958 - loss: 0.1013 - val_accuracy: 0.5000 - val_loss: 0.7244
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 326ms/step - accuracy: 1.0000 - loss: 0.0471 - val_accuracy: 0.5000 - val_loss: 0.7107
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 326ms/step - accuracy: 1.0000 - loss: 0.0270 - val_accuracy: 0.5000 - val_loss: 0.7208
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 328ms/step - accuracy: 1.0000 - loss: 0.0091 - val_accuracy: 0.5000 - val_loss: 0

In [41]:
model_ResNet10.predict(X_val_array_resampled)[0:20]

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 208ms/step


array([[0.7569167 , 0.24308333],
       [0.75380856, 0.24619144],
       [0.7734363 , 0.2265637 ],
       [0.73854095, 0.26145902],
       [0.7827304 , 0.21726963],
       [0.76103044, 0.23896958],
       [0.7624994 , 0.23750056],
       [0.75138694, 0.24861306],
       [0.7599778 , 0.24002218],
       [0.7732955 , 0.2267044 ],
       [0.76910627, 0.2308937 ],
       [0.7638732 , 0.2361268 ],
       [0.7767476 , 0.22325242],
       [0.7504792 , 0.24952078],
       [0.74204266, 0.25795737],
       [0.7424291 , 0.25757092],
       [0.782885  , 0.21711498],
       [0.72948205, 0.27051795],
       [0.75849605, 0.24150394],
       [0.7581919 , 0.24180807]], dtype=float32)

### ResNet101: Try more complex model

In [14]:
ROWS = 224; COLS = 224; CHANNELS = 43; CLASSES = 2
block_layers = [3,4,23,3]
# Build Network Graph 
model_ResNet101 = ResNet(shape = (ROWS, COLS, CHANNELS), classes = CLASSES, block_layers = block_layers)
# Compile Model 
l_rate = 1.e-4
opt = keras.optimizers.Adam(learning_rate=l_rate)
model_ResNet101.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
# Train Model 
batch = 64; epochs = 10
start_time = time.time()
history_ResNet101 = model_ResNet101.fit(X_train_array_resampled, y_train_resampled_one_hot, 
                                      epochs = epochs, batch_size = batch,
                                     validation_data = (X_val_array_resampled, y_val_resampled_one_hot))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nElapsed time: {elapsed_time} seconds")

2024-12-15 13:09:38.584236: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3676495872 exceeds 10% of free system memory.
2024-12-15 13:09:40.307908: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3676495872 exceeds 10% of free system memory.


Epoch 1/10


I0000 00:00:1734268211.511209  667327 service.cc:146] XLA service 0x7f1998002720 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734268211.511251  667327 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-12-15 13:10:12.663400: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-15 13:10:16.276678: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90300
I0000 00:00:1734268244.120297  667327 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7/7 ━━━━━━━━━━━━━━━━━━━━ 102s 6s/step - accuracy: 0.5278 - loss: 3.4380 - val_accuracy: 0.5000 - val_loss: 0.7022
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 750ms/step - accuracy: 0.5908 - loss: 1.3838 - val_accuracy: 0.5000 - val_loss: 0.7010
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 754ms/step - accuracy: 0.7173 - loss: 0.7623 - val_accuracy: 0.5000 - val_loss: 0.6945
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 756ms/step - accuracy: 0.8415 - loss: 0.4475 - val_accuracy: 0.5000 - val_loss: 0.7086
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 758ms/step - accuracy: 0.9358 - loss: 0.3303 - val_accuracy: 0.5000 - val_loss: 0.7278
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 760ms/step - accuracy: 0.9183 - loss: 0.2453 - val_accuracy: 0.5000 - val_loss: 0.7110
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 761ms/step - accuracy: 0.9913 - loss: 0.1156 - val_accuracy: 0.5000 - val_loss: 0.7169
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 758ms/step - accuracy: 1.0000 - loss: 0.0478 - val_accuracy: 0.5000 - val_loss: 0.6993
Epoc

In [18]:
model_ResNet101.predict(X_val_array_resampled)[0:20]

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step


array([[0.63307756, 0.3669224 ],
       [0.59208673, 0.4079133 ],
       [0.5898937 , 0.41010627],
       [0.5683737 , 0.43162635],
       [0.6350663 , 0.3649337 ],
       [0.6089314 , 0.3910686 ],
       [0.6027144 , 0.3972856 ],
       [0.6252898 , 0.37471014],
       [0.6122896 , 0.38771036],
       [0.62071234, 0.37928772],
       [0.60527813, 0.39472184],
       [0.60391414, 0.3960858 ],
       [0.62466466, 0.37533534],
       [0.5985049 , 0.40149507],
       [0.6146456 , 0.38535446],
       [0.5900499 , 0.40995008],
       [0.630843  , 0.36915702],
       [0.5908219 , 0.40917808],
       [0.59003   , 0.40997002],
       [0.5835712 , 0.41642877]], dtype=float32)

In [20]:
model_ResNet101.evaluate(X_train_array_resampled, y_train_resampled_one_hot)

14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 312ms/step - accuracy: 0.5237 - loss: 0.7004


[0.7043793201446533, 0.5]

#### COMMENTS

Even with Correct Label Encoding, and different complexities of ResNet, and balanced datasets (downsampling), and different number of channels (3 vs. 43), the model only predicts one class.

Note: When using 43 Channels, the probabilities get closer to 0.5, while when using 3 channels, the probabilities are close to 0.9

### Increase Model complexity: ResNet 101